In [1]:
import yaml
from pathlib import Path
import pandas as pd

In [2]:
def get_total_volume(row: pd.Series) -> int:
    total_vol = 0
    for s in sizes.size_name.values:
        total_vol += row[s] * sizes.loc[sizes.size_name==s, "volume_m3"].values[0]
    return total_vol

In [3]:
path_root = Path.cwd().parent

In [4]:
# サイズ一覧読み込み
sizes = pd.read_csv(path_root/"config/box_size.csv")
# 体積（m3）のカラムを追加
sizes["volume_m3"] = sizes.width * sizes.height * sizes.depth / 1000000000

In [5]:
# RBPのサイズ読み込み
with open(path_root/"config/config.yaml","r") as s:
    config = yaml.safe_load(s)
rbp_volume = (config["rbp"]["width"] * config["rbp"]["height"] * config["rbp"]["depth"]) / 1000000000

In [16]:
# get list of result files
files = [f for f in Path.iterdir(path_root/"results") if f.name.endswith("csv")]

In [6]:
for f in files:
    data = pd.read_csv(f, index_col=[0])
    data["total_volume"] = data.apply(get_total_volume, axis=1)
    data["load_factor_perc"] = data.total_volume/rbp_volume * 100

    # update file with load factor column
    data.to_csv(f)